# Scraping

This aims to retrieve Swiss-related subreddits from the suggestions in /r/Switzerland

NOTE: We could have simply copied the names from the website, as they are not an unbereable amount to be done by hand, but we thought this would be an OK warm up exercise to touch more different thing of the typical Data Analysis pipeline in our project.

In [2]:
## Importing libraries
import requests
from bs4 import BeautifulSoup
import numpy as np

### Getting the list and Saving to CSV

'Requests' library is used to get the HTML data using a base url. 'BeautifulSoup' is then used to parse the HTML.

In [3]:
form_url = 'https://www.reddit.com/r/Switzerland/'
r = requests.get(form_url)
soup = BeautifulSoup(r.text, 'html.parser')

We have found a problems with being mistaken by a bot because we ran this several times consecutively!

In [60]:
#The objective is to come up with a SubReddits' name list
SubReddits = list()

#First we define the categories we are interested in, to avoid including links related to switzerland because of
# neighbouring countries, etc.
list_of_interesting_categories = ['» Other general Swiss Subreddits','» Special Interest Swiss Subreddits', '» Universities and Institutions']

# Then we loop in search for our links
for bq in soup.find_all('blockquote'):
    if bq.h3.text in list_of_interesting_categories:
        
        #getting all the links
        for link in bq.find_all('a', href=True):    
            parts = link['href'].split("/")
            if len(parts) == 3:
                SubReddits.append(parts[2])
    
print(SubReddits)                

['AskSwitzerland', 'Basel', 'Bern', 'BielBienne', 'Buenzli', 'Frauenfeld', 'Fribourg', 'Geneva', 'Liestal', 'Luzern', 'Morcote', 'Neuchatel', 'Schaffhausen', 'SanktGallen', 'Schwiiz', 'Solothurn', 'Stans', 'Suisse', 'Thun', 'Ticino', 'Winterthur', 'Zermatt', 'Zug', 'Zurich', 'Breitling', 'CHTrees', 'FCBasel', 'MatterhornPorn', 'Migros', 'Schweiz', 'SwissArmy', 'SwissArmyKnives', 'SwissBuyers', 'SwissGaming', 'SwissGuns', 'SwissHockey', 'SwissESports', 'SwissMountainDogs', 'SwissNews', 'SwissProblems', 'SwissRap', 'SwissSuperLeague', 'SwissHistory', 'CERN', 'EPFL', 'ETHZ', 'UZH']


In [65]:
# We save the list to csv for later use
np.savetxt("Swiss_SR.csv", SubReddits, delimiter=",", fmt='%s', header='Swiss SubReddit Names')